In [130]:
import os
import pandas as pd

# Chemin du dossier à analyser
dossier = 'data/raw/'
liste_fichiers = []

for fichier in os.listdir(dossier):
    chemin_complet = os.path.join(dossier, fichier)
    if os.path.isfile(chemin_complet):
        nom_sans_ext = os.path.splitext(fichier)[0]
        if len(nom_sans_ext) >= 4 and nom_sans_ext[-4:].isdigit():
            annee = int(nom_sans_ext[-4:])
            nom_restant = nom_sans_ext[:-4]
        else:
            annee = None
            nom_restant = nom_sans_ext

        try:
            df_listing_temp = pd.read_csv(chemin_complet, nrows=0, encoding='ISO-8859-1')
            headers = list(df_listing_temp.columns)
        except Exception as e:
            print(f"⚠️ Erreur lecture header pour {fichier} : {e}")
            headers = []

        liste_fichiers.append({
            'fichier': fichier,
            'annee': annee,
            'fichier_categorie': nom_restant,
            'path': chemin_complet,
            'header': headers
        })

df_listing = pd.DataFrame(liste_fichiers)
df_listing = df_listing.dropna(subset=['annee'])
df_listing['annee'] = df_listing['annee'].astype(int)
print(df_listing.head())

                     fichier  annee  fichier_categorie  \
0            caract-2023.csv   2023            caract-   
1  caracteristiques-2017.csv   2017  caracteristiques-   
2  caracteristiques-2018.csv   2018  caracteristiques-   
3  caracteristiques-2019.csv   2019  caracteristiques-   
4  caracteristiques-2020.csv   2020  caracteristiques-   

                                 path  \
0            data/raw/caract-2023.csv   
1  data/raw/caracteristiques-2017.csv   
2  data/raw/caracteristiques-2018.csv   
3  data/raw/caracteristiques-2019.csv   
4  data/raw/caracteristiques-2020.csv   

                                              header  
0  [Num_Acc;"jour";"mois";"an";"hrmn";"lum";"dep"...  
1  [Num_Acc, an, mois, jour, hrmn, lum, agg, int,...  
2  [Num_Acc, an, mois, jour, hrmn, lum, agg, int,...  
3  [Num_Acc;"jour";"mois";"an";"hrmn";"lum";"dep"...  
4  [Num_Acc;"jour";"mois";"an";"hrmn";"lum";"dep"...  


In [131]:
df_listing[df_listing['annee'] == 2020]

,fichier,annee,fichier_categorie,path,header
4,caracteristiques-2020.csv,2020,caracteristiques-,data/raw/caracteristiques-2020.csv,"[Num_Acc;""jour"";""mois"";""an"";""hrmn"";""lum"";""dep""..."
22,lieux-2020.csv,2020,lieux-,data/raw/lieux-2020.csv,"[Num_Acc;""catr"";""voie"";""v1"";""v2"";""circ"";""nbv"";..."
41,usagers-2020.csv,2020,usagers-,data/raw/usagers-2020.csv,"[Num_Acc;""id_vehicule"";""num_veh"";""place"";""catu..."
60,vehicules-2020.csv,2020,vehicules-,data/raw/vehicules-2020.csv,"[Num_Acc;""id_vehicule"";""num_veh"";""senc"";""catv""..."


In [132]:
#homogénéisation de la colonne fichier_categorie
df_listing['fichier_categorie'] = df_listing['fichier_categorie'].replace('lieux-', 'lieux_')
df_listing['fichier_categorie'] = df_listing['fichier_categorie'].replace('usagers-', 'usagers_')
df_listing['fichier_categorie'] = df_listing['fichier_categorie'].replace('vehicules-', 'vehicules_')
df_listing.loc[df_listing['fichier_categorie'].isin(['caract-', 'caracteristiques-', 'carcteristiques-']), 'fichier_categorie'] = 'caracteristiques_'
df_listing.loc[df_listing['fichier_categorie'].isin(['lieux-', 'caracteristiques-', 'carcteristiques']), 'fichier_categorie'] = 'caracteristiques_'
print(df_listing['fichier_categorie'].drop_duplicates())

0     caracteristiques_
19               lieux_
38             usagers_
57           vehicules_
Name: fichier_categorie, dtype: object


In [133]:
print(df_listing['header'].drop_duplicates())

0     [Num_Acc;"jour";"mois";"an";"hrmn";"lum";"dep"...
1     [Num_Acc, an, mois, jour, hrmn, lum, agg, int,...
9     [Num_Acc\tan\tmois\tjour\thrmn\tlum\tagg\tint\...
18    [Accident_Id;"jour";"mois";"an";"hrmn";"lum";"...
19    [Num_Acc, catr, voie, v1, v2, circ, nbv, pr, p...
21    [Num_Acc;"catr";"voie";"v1";"v2";"circ";"nbv";...
38    [Num_Acc, place, catu, grav, sexe, trajet, sec...
40    [Num_Acc;"id_vehicule";"num_veh";"place";"catu...
42    [Num_Acc;"id_usager";"id_vehicule";"num_veh";"...
57    [Num_Acc, senc, catv, occutc, obs, obsm, choc,...
59    [Num_Acc;"id_vehicule";"num_veh";"senc";"catv"...
Name: header, dtype: object


In [134]:
#enregistrement du listing de fichier + header
# 
df_listing.to_csv("data/processed/listingfiles.csv", index=False,)

In [210]:
import pandas as pd

def get_data(Annee, Cat, df_listing):
    ligne = df_listing.loc[(df_listing["annee"] == int(Annee)) & (df_listing["fichier_categorie"] == Cat), "path"]
    header = df_listing.loc[(df_listing["annee"] == int(Annee)) & (df_listing["fichier_categorie"] == Cat), "header"]
    
    if not ligne.empty:
        chemin = ligne.iloc[0]
        header_val = header.iloc[0]
        
       
       
        # Si header_val est une chaîne de colonnes séparées, on teste le séparateur sur cette chaîne
        if ';' in header_val[0]:
            sep = ';'
        elif '\t' in header_val:
            sep = '\t'
        else:
            sep = ','
        print("sep=",sep)
        for enc in ['utf-8', 'cp1252', 'iso-8859-1','latin-1']:
            print(chemin, sep, enc)
            try:
                print(f"Essai avec encodage : {enc}")
                """
                 # Lecture brute des premières lignes pour détecter les anomalies (ex: guillemets doubles)
                with open(chemin, 'r', encoding=enc, errors='replace') as f:
                    preview = [next(f) for _ in range(10)]
                preview_text = ''.join(preview)
                contient_guillemets_doubles = '""' in preview_text

                # Si anomalie détectée, nettoyage manuel ligne par ligne
                if contient_guillemets_doubles:
                    print("⚠️ Fichier suspect, tentative de nettoyage manuel")
                    with open(chemin, 'r', encoding=enc, errors='replace') as f:
                        lignes_nettoyees = [ligne.replace('""', '"').strip() for ligne in f]
                    contenu_nettoye = StringIO('\n'.join(lignes_nettoyees))
                    df_listing_read = pd.read_csv(contenu_nettoye, sep=sep, quotechar='"', low_memory=False)
                    df_listing_read.columns = df_listing_read.columns.str.replace('"', '').str.strip()
                else:
                    # Lecture standard si aucun problème détecté
                """
                df_listing_read = pd.read_csv(chemin, sep=sep, encoding=enc, low_memory=False)
                print(chemin, sep, enc)
                print(f"✅ Lecture réussie avec {enc}")
                return df_listing_read
            
            except FileNotFoundError:
                print(f"    ❌ Fichier non trouvé : {chemin}")
                return pd.DataFrame()
            except UnicodeDecodeError as e:
                print(f"    ⚠️ Échec avec {enc} : {e}")
            except Exception as e:
                print(f"    ❌ Autre erreur avec {enc} : {e}")
        
        # Si aucun encodage ne marche
        print(f"❌ Impossible de lire le fichier {chemin} avec les encodages testés.")
        return pd.DataFrame()
    else:
        print(f"❌ Aucune configuration trouvée pour année={Annee}, catégorie={Cat}")
        return pd.DataFrame()

In [211]:
def get_annee_dispo(annees_list, annee_min, annee_max):
    """
    trie une liste d'année en tenant compte d'un mini et d'un maxi
    ex: print(get_annee_dispo(df['annee'],'2020','2021'))
    """
    import pandas as pd

    def safe_int(value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return None

    annee_min = safe_int(annee_min)
    annee_max = safe_int(annee_max)

    # Nettoyage ici, sur la série brute
    annees = pd.to_numeric(annees_list, errors='coerce').dropna().astype(int)

    if annee_min is not None and annee_max is not None:
        annees = annees[annees.between(annee_min, annee_max)]
    elif annee_min is not None:
        annees = annees[annees >= annee_min]
    elif annee_max is not None:
        annees = annees[annees <= annee_max]

    return annees.drop_duplicates().sort_values(ascending=False).reset_index(drop=True)

# Utilisation
#print(list_annee)


In [212]:
df_listing.head(15)

,fichier,annee,fichier_categorie,path,header
0,caract-2023.csv,2023,caracteristiques_,data/raw/caract-2023.csv,"[Num_Acc;""jour"";""mois"";""an"";""hrmn"";""lum"";""dep""..."
1,caracteristiques-2017.csv,2017,caracteristiques_,data/raw/caracteristiques-2017.csv,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,..."
2,caracteristiques-2018.csv,2018,caracteristiques_,data/raw/caracteristiques-2018.csv,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,..."
3,caracteristiques-2019.csv,2019,caracteristiques_,data/raw/caracteristiques-2019.csv,"[Num_Acc;""jour"";""mois"";""an"";""hrmn"";""lum"";""dep""..."
4,caracteristiques-2020.csv,2020,caracteristiques_,data/raw/caracteristiques-2020.csv,"[Num_Acc;""jour"";""mois"";""an"";""hrmn"";""lum"";""dep""..."
5,caracteristiques_2005.csv,2005,caracteristiques_,data/raw/caracteristiques_2005.csv,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,..."
6,caracteristiques_2006.csv,2006,caracteristiques_,data/raw/caracteristiques_2006.csv,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,..."
7,caracteristiques_2007.csv,2007,caracteristiques_,data/raw/caracteristiques_2007.csv,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,..."
8,caracteristiques_2008.csv,2008,caracteristiques_,data/raw/caracteristiques_2008.csv,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,..."
9,caracteristiques_2009.csv,2009,caracteristiques_,data/raw/caracteristiques_2009.csv,[Num_Acc\tan\tmois\tjour\thrmn\tlum\tagg\tint\...


In [213]:
annee_dispo = get_annee_dispo(df_listing['annee'], 2016, 2022)

In [214]:
get_annee_dispo(df_listing['annee'], 2016, 2022)

0    2022
1    2021
2    2020
3    2019
4    2018
5    2017
6    2016
Name: annee, dtype: int32

In [215]:
get_data(2020, 'caracteristiques_', df_listing)

sep= ;
data/raw/caracteristiques-2020.csv ; utf-8
Essai avec encodage : utf-8
data/raw/caracteristiques-2020.csv ; utf-8
✅ Lecture réussie avec utf-8


,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,202000000001,7,3,2020,16:55,1,91,91657,2,3,1,3,HENRI BARBUSSE (AVENUE),"48,7053500","2,4384100"
1,202000000002,7,3,2020,08:35,2,91,91657,2,9,7,6,MOUSSEAUX(CHEMIN),"48,6900000","2,4100000"
2,202000000003,7,3,2020,13:30,1,91,91174,2,2,1,3,CARNOT(AVENUE),"48,6106700","2,4758200"
3,202000000004,7,3,2020,18:50,5,91,91215,2,1,1,6,VICTOR HUGO (AVENUE),"48,6978200","2,5244600"
4,202000000005,7,3,2020,11:00,1,77,77181,1,6,1,2,LAGNY (RUE DE ) - D35,"48,8286457","2,7059707"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47739,202000047740,30,11,2020,07:50,1,13,13028,2,1,1,4,PUITS DE BRUNET (CHEMIN DU),"43,1865750","5,6003810"
47740,202000047741,30,11,2020,17:00,2,06,06083,2,1,9,3,CASTAGNINS (ROUTE DES),"43,7799570","7,4818980"
47741,202000047742,1,12,2020,10:00,1,06,06069,2,1,1,5,MADELEINE,"43,6571157","6,9337155"
47742,202000047743,28,11,2020,11:45,1,05,05061,1,1,1,2,Bernard GIVAUDAN (Avenue) D900B,"44,5406800","6,0641900"


In [217]:
all_accidents = []
from io import StringIO

# Récupérer les années disponibles dans la colonne 'annee'
annee_dispo = get_annee_dispo(df_listing['annee'], 2004, 2023)

for annee in annee_dispo:
    # On ouvre les fichiers
    print(f"\nTraitement des fichiers pour l'année {annee}...")
    caract = get_data(annee, 'caracteristiques_', df_listing)
    lieux = get_data(annee, 'lieux_', df_listing)
    usagers= get_data(annee, 'usagers_', df_listing)
    vehicules= get_data(annee, 'vehicules_', df_listing)
    # homogenisation colonne Num_acc
    if 'Accident_Id' in caract.columns:
        caract = caract.rename(columns={'Accident_Id': 'Num_Acc'})
    try:
        # merge des différentes tables avec gestion des erreurs 
        merged= (usagers
            .merge(lieux, on='Num_Acc')
            .merge(vehicules, on=['Num_Acc', 'id_vehicule', 'num_veh'], how='inner')
            .merge(caract, on='Num_Acc')
        )
        all_accidents.append(merged)
        print(f"    ✅Fusion réussie pour l'année {annee}")
    except KeyError as e:
        print(f"    ⚠️ Clé manquante pour la fusion avec id_vehicule : {e}")
    # Tentative 2 : fusion sans id_vehicule
    try:
        merged = (usagers
                    .merge(lieux, on='Num_Acc')
                    .merge(vehicules, on=['Num_Acc', 'num_veh'], how='inner')
                    .merge(caract, on='Num_Acc')
                    )
        all_accidents.append(merged)
        print(f"    ✅Fusion réussie SANS id_vehicule pour l'année {annee}")


    except KeyError as e:
        print(f"    Erreur : colonne manquante {e}")
    except Exception as e:
        print(f"    Erreur lors de la fusion : {e}")  
    print(f"accident{annee}")
#On concatène    
if  all_accidents: 
    combined_data = pd.concat(all_accidents, ignore_index=True)
    #enregistrement du fichier final
    combined_data.to_csv('data/processed/Accident_auto_combined.csv', index=False, encoding='utf-8')
    print(f"Combined export: {len(combined_data)} total records across all years")
else: print(f"Combined export: no record")
#pd.DataFrame([all_accidents]).to_csv('Accident_auto.csv', index=False, encoding='utf-8')  # ou 'ISO-8859-1' si besoin


Traitement des fichiers pour l'année 2023...
sep= ;
data/raw/caract-2023.csv ; utf-8
Essai avec encodage : utf-8
data/raw/caract-2023.csv ; utf-8
✅ Lecture réussie avec utf-8
sep= ;
data/raw/lieux-2023.csv ; utf-8
Essai avec encodage : utf-8
data/raw/lieux-2023.csv ; utf-8
✅ Lecture réussie avec utf-8
sep= ;
data/raw/usagers-2023.csv ; utf-8
Essai avec encodage : utf-8
data/raw/usagers-2023.csv ; utf-8
✅ Lecture réussie avec utf-8
sep= ;
data/raw/vehicules-2023.csv ; utf-8
Essai avec encodage : utf-8
data/raw/vehicules-2023.csv ; utf-8
✅ Lecture réussie avec utf-8
    ✅Fusion réussie pour l'année 2023
    ✅Fusion réussie SANS id_vehicule pour l'année 2023
accident2023

Traitement des fichiers pour l'année 2022...
sep= ;
data/raw/carcteristiques-2022.csv ; utf-8
Essai avec encodage : utf-8
data/raw/carcteristiques-2022.csv ; utf-8
✅ Lecture réussie avec utf-8
sep= ;
data/raw/lieux-2022.csv ; utf-8
Essai avec encodage : utf-8
data/raw/lieux-2022.csv ; utf-8
✅ Lecture réussie avec utf-8
